# Dog Breed Prediction


in this project we will see how to use keras and TensorFlow to build train, and test a Convolutional Neural Network capable of identifying the breed of a dog in a supplied image. This is a supervised learning problem , specifically a multiclass classification problem.

In [2]:
# Run this cell and select the kaggle.json file downloaded
# From the Kaggle account setting page.
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mahoorshams","key":"2b5ec3a9e9700362b646e3b095c12b8d"}'}

we will start by connection to Kaggle API which can be downloaded from your Kaggle account's setting and uploading it here(upload box)

In [3]:
# Next, install the Kaggle API client.
!pip install -q kaggle

Next we will install Kaggle API using pip installation.

In [5]:
# The Kaggle API client expects this file to be in ~/.kaggle, so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permission avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

 Setting up Kaggle using Kaggle API.

In [6]:
# Creating directory and changing the current working directory
!mkdir dog_dataset
%cd dog_dataset

/content/dog_dataset
